In [1]:
import sqlalchemy

In [2]:
sqlalchemy.__version__

'1.1.13'

In [3]:
from sqlalchemy import create_engine

In [4]:
engine = create_engine('mysql+pymysql://root@127.0.0.1:3306/demo', echo=True)

In [5]:
engine.name

'mysql'

In [6]:
from sqlalchemy.ext.declarative import declarative_base

In [7]:
Base = declarative_base()

In [9]:
from sqlalchemy import Column, Integer, String

In [22]:
class User(Base):
    __tablename__ = 'user'
    
    id = Column(Integer, autoincrement=True, primary_key=True)
    name = Column(String(64), unique=True, nullable=False)
    age = Column(Integer)
    
    def __str__(self):
        return 'User(id={}, name={}, age={})'.format(self.id, self.name, self.age)
    

/Users/miracleYoung/.pyenv/versions/3.6.1/envs/miracle_3.6/lib/python3.6/site-packages/sqlalchemy/ext/declarative/clsregistry.py:120: SAWarning: This declarative base already contains a class with the same class name and module name as __main__.User, and will be replaced in the string-lookup table.
  item.__name__


InvalidRequestError: Table 'user' is already defined for this MetaData instance.  Specify 'extend_existing=True' to redefine options and columns on an existing Table object.

In [15]:
Base.metadata.create_all(engine)

2017-11-26 09:30:45,968 INFO sqlalchemy.engine.base.Engine DESCRIBE `user`
2017-11-26 09:30:45,970 INFO sqlalchemy.engine.base.Engine {}


In [13]:
Base.metadata.drop_all(engine)

2017-11-26 09:29:22,928 INFO sqlalchemy.engine.base.Engine DESCRIBE `user`
2017-11-26 09:29:22,929 INFO sqlalchemy.engine.base.Engine {}
2017-11-26 09:29:22,933 INFO sqlalchemy.engine.base.Engine 
DROP TABLE user
2017-11-26 09:29:22,934 INFO sqlalchemy.engine.base.Engine {}
2017-11-26 09:29:22,936 INFO sqlalchemy.engine.base.Engine COMMIT


In [16]:
from sqlalchemy.orm import sessionmaker # 一个session 就是一个连接

In [17]:
Session = sessionmaker(bind=engine)

In [18]:
session = Session()

In [19]:
user = User()

In [20]:
print(user)

User(id=None, name=None, age=None)


In [23]:
user.name = 'miracle'

In [25]:
user.age = 18

In [26]:
print(user)

User(id=None, name=miracle, age=18)


In [27]:
session.add(user)

In [28]:
session.commit()

2017-11-26 09:50:51,743 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2017-11-26 09:50:51,745 INFO sqlalchemy.engine.base.Engine INSERT INTO user (name, age) VALUES (%(name)s, %(age)s)
2017-11-26 09:50:51,746 INFO sqlalchemy.engine.base.Engine {'name': 'miracle', 'age': 18}
2017-11-26 09:50:51,748 INFO sqlalchemy.engine.base.Engine COMMIT


In [29]:
print(user)

2017-11-26 09:51:32,381 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2017-11-26 09:51:32,382 INFO sqlalchemy.engine.base.Engine SELECT user.id AS user_id, user.name AS user_name, user.age AS user_age 
FROM user 
WHERE user.id = %(param_1)s
2017-11-26 09:51:32,383 INFO sqlalchemy.engine.base.Engine {'param_1': 1}
User(id=1, name=miracle, age=18)


In [30]:
print(user)

User(id=1, name=miracle, age=18)


- 创建Session类
- 初始化session 实例
- 实例化Model
- 修改我们的实例属性
- 把实例加入到session中
- session commit

In [32]:
user._sa_instance_state.dict

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState at 0x10c7faba8>,
 'age': 18,
 'id': 1,
 'name': 'miracle'}

In [33]:
user2 = User()

In [34]:
user2.name = 'miracle'

In [35]:
user2.age = 30

In [36]:
session.add(user2)

In [37]:
session.commit()

2017-11-26 10:02:47,625 INFO sqlalchemy.engine.base.Engine INSERT INTO user (name, age) VALUES (%(name)s, %(age)s)
2017-11-26 10:02:47,627 INFO sqlalchemy.engine.base.Engine {'name': 'miracle', 'age': 30}
2017-11-26 10:02:47,628 INFO sqlalchemy.engine.base.Engine ROLLBACK


IntegrityError: (pymysql.err.IntegrityError) (1062, "Duplicate entry 'miracle' for key 'name'") [SQL: 'INSERT INTO user (name, age) VALUES (%(name)s, %(age)s)'] [parameters: {'name': 'miracle', 'age': 30}]

In [38]:
user3 = User()

In [39]:
user3.name = 'young'

In [40]:
user3.age = 50

In [41]:
session.add(user3)

In [42]:
session.commit()

InvalidRequestError: This Session's transaction has been rolled back due to a previous exception during flush. To begin a new transaction with this Session, first issue Session.rollback(). Original exception was: (pymysql.err.IntegrityError) (1062, "Duplicate entry 'miracle' for key 'name'") [SQL: 'INSERT INTO user (name, age) VALUES (%(name)s, %(age)s)'] [parameters: {'name': 'miracle', 'age': 30}]

**一旦发生commit 异常后，后续的任何操作都无法执行，必须先执行rollback**

In [43]:
session.rollback()

/Users/miracleYoung/.pyenv/versions/3.6.1/envs/miracle_3.6/lib/python3.6/site-packages/sqlalchemy/orm/session.py:507: SAWarning: Session's state has been changed on a non-active transaction - this state will be discarded.
  "Session's state has been changed on "


In [44]:
session.add(user3)

In [45]:
session.commit()

2017-11-26 10:05:07,073 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2017-11-26 10:05:07,075 INFO sqlalchemy.engine.base.Engine INSERT INTO user (name, age) VALUES (%(name)s, %(age)s)
2017-11-26 10:05:07,076 INFO sqlalchemy.engine.base.Engine {'name': 'young', 'age': 50}
2017-11-26 10:05:07,077 INFO sqlalchemy.engine.base.Engine COMMIT


In [ ]:
try:
    session.commit()
except Exception as e:
    session.rollback()
    raise e

In [46]:
print(user)

2017-11-26 10:33:12,783 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2017-11-26 10:33:12,784 INFO sqlalchemy.engine.base.Engine SELECT user.id AS user_id, user.name AS user_name, user.age AS user_age 
FROM user 
WHERE user.id = %(param_1)s
2017-11-26 10:33:12,786 INFO sqlalchemy.engine.base.Engine {'param_1': 1}
User(id=1, name=miracle, age=18)


In [47]:
user.age = 20

In [48]:
session.add(user)

In [49]:
session.commit()

2017-11-26 10:33:33,156 INFO sqlalchemy.engine.base.Engine UPDATE user SET age=%(age)s WHERE user.id = %(user_id)s
2017-11-26 10:33:33,157 INFO sqlalchemy.engine.base.Engine {'age': 20, 'user_id': 1}
2017-11-26 10:33:33,158 INFO sqlalchemy.engine.base.Engine COMMIT


In [52]:
user._sa_instance_state.dict

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState at 0x10c7faba8>,
 'age': 20,
 'id': 1,
 'name': 'miracle'}

In [53]:
u = User()

In [54]:
u._sa_instance_state.dict

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState at 0x10c5fabe0>}

In [55]:
u.id = 1

In [56]:
u.name = 'miracle'

In [57]:
u.age = 17

In [58]:
session.add(u)

In [60]:
session.rollback()

In [61]:
u = User()

In [62]:
session.delete(user3)

In [63]:
session.commit()

2017-11-26 10:37:01,850 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2017-11-26 10:37:01,852 INFO sqlalchemy.engine.base.Engine SELECT user.id AS user_id, user.name AS user_name, user.age AS user_age 
FROM user 
WHERE user.id = %(param_1)s
2017-11-26 10:37:01,853 INFO sqlalchemy.engine.base.Engine {'param_1': 3}
2017-11-26 10:37:01,854 INFO sqlalchemy.engine.base.Engine DELETE FROM user WHERE user.id = %(id)s
2017-11-26 10:37:01,855 INFO sqlalchemy.engine.base.Engine {'id': 3}
2017-11-26 10:37:01,856 INFO sqlalchemy.engine.base.Engine COMMIT


In [64]:
print(u)

User(id=None, name=None, age=None)


In [65]:
session.delete(u)

InvalidRequestError: Instance '<User at 0x10c6d3c18>' is not persisted

In [66]:
u._sa_instance_state.dict

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState at 0x10c6d3e48>}

In [67]:
session.rollback()

In [68]:
q = session.query(User)

In [69]:
q

In [71]:
for u in q:
    print(u)

2017-11-26 10:43:03,526 INFO sqlalchemy.engine.base.Engine SELECT user.id AS user_id, user.name AS user_name, user.age AS user_age 
FROM user
2017-11-26 10:43:03,527 INFO sqlalchemy.engine.base.Engine {}
User(id=1, name=miracle, age=20)
User(id=5, name=test, age=19)


In [72]:
import random

In [73]:
for x in range(100):
    user = User()
    user.name = 'user_{}'.format(x)
    user.age = random.randint(0, 100)
    session.add(user)
session.commit()

2017-11-26 10:43:24,531 INFO sqlalchemy.engine.base.Engine INSERT INTO user (name, age) VALUES (%(name)s, %(age)s)
2017-11-26 10:43:24,532 INFO sqlalchemy.engine.base.Engine {'name': 'user_0', 'age': 50}
2017-11-26 10:43:24,533 INFO sqlalchemy.engine.base.Engine INSERT INTO user (name, age) VALUES (%(name)s, %(age)s)
2017-11-26 10:43:24,534 INFO sqlalchemy.engine.base.Engine {'name': 'user_1', 'age': 4}
2017-11-26 10:43:24,535 INFO sqlalchemy.engine.base.Engine INSERT INTO user (name, age) VALUES (%(name)s, %(age)s)
2017-11-26 10:43:24,535 INFO sqlalchemy.engine.base.Engine {'name': 'user_2', 'age': 24}
2017-11-26 10:43:24,537 INFO sqlalchemy.engine.base.Engine INSERT INTO user (name, age) VALUES (%(name)s, %(age)s)
2017-11-26 10:43:24,537 INFO sqlalchemy.engine.base.Engine {'name': 'user_3', 'age': 46}
2017-11-26 10:43:24,538 INFO sqlalchemy.engine.base.Engine INSERT INTO user (name, age) VALUES (%(name)s, %(age)s)
2017-11-26 10:43:24,539 INFO sqlalchemy.engine.base.Engine {'name': 'u

2017-11-26 10:43:24,588 INFO sqlalchemy.engine.base.Engine {'name': 'user_40', 'age': 50}
2017-11-26 10:43:24,589 INFO sqlalchemy.engine.base.Engine INSERT INTO user (name, age) VALUES (%(name)s, %(age)s)
2017-11-26 10:43:24,590 INFO sqlalchemy.engine.base.Engine {'name': 'user_41', 'age': 47}
2017-11-26 10:43:24,590 INFO sqlalchemy.engine.base.Engine INSERT INTO user (name, age) VALUES (%(name)s, %(age)s)
2017-11-26 10:43:24,591 INFO sqlalchemy.engine.base.Engine {'name': 'user_42', 'age': 29}
2017-11-26 10:43:24,592 INFO sqlalchemy.engine.base.Engine INSERT INTO user (name, age) VALUES (%(name)s, %(age)s)
2017-11-26 10:43:24,592 INFO sqlalchemy.engine.base.Engine {'name': 'user_43', 'age': 49}
2017-11-26 10:43:24,593 INFO sqlalchemy.engine.base.Engine INSERT INTO user (name, age) VALUES (%(name)s, %(age)s)
2017-11-26 10:43:24,594 INFO sqlalchemy.engine.base.Engine {'name': 'user_44', 'age': 0}
2017-11-26 10:43:24,594 INFO sqlalchemy.engine.base.Engine INSERT INTO user (name, age) VAL

2017-11-26 10:43:24,642 INFO sqlalchemy.engine.base.Engine {'name': 'user_80', 'age': 59}
2017-11-26 10:43:24,643 INFO sqlalchemy.engine.base.Engine INSERT INTO user (name, age) VALUES (%(name)s, %(age)s)
2017-11-26 10:43:24,644 INFO sqlalchemy.engine.base.Engine {'name': 'user_81', 'age': 36}
2017-11-26 10:43:24,645 INFO sqlalchemy.engine.base.Engine INSERT INTO user (name, age) VALUES (%(name)s, %(age)s)
2017-11-26 10:43:24,645 INFO sqlalchemy.engine.base.Engine {'name': 'user_82', 'age': 58}
2017-11-26 10:43:24,646 INFO sqlalchemy.engine.base.Engine INSERT INTO user (name, age) VALUES (%(name)s, %(age)s)
2017-11-26 10:43:24,647 INFO sqlalchemy.engine.base.Engine {'name': 'user_83', 'age': 40}
2017-11-26 10:43:24,648 INFO sqlalchemy.engine.base.Engine INSERT INTO user (name, age) VALUES (%(name)s, %(age)s)
2017-11-26 10:43:24,648 INFO sqlalchemy.engine.base.Engine {'name': 'user_84', 'age': 67}
2017-11-26 10:43:24,649 INFO sqlalchemy.engine.base.Engine INSERT INTO user (name, age) VA

In [74]:
q = q.filter(User.age < 18)

In [75]:
for u in q:
    print(u)

2017-11-26 10:45:43,884 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2017-11-26 10:45:43,886 INFO sqlalchemy.engine.base.Engine SELECT user.id AS user_id, user.name AS user_name, user.age AS user_age 
FROM user 
WHERE user.age < %(age_1)s
2017-11-26 10:45:43,887 INFO sqlalchemy.engine.base.Engine {'age_1': 18}
User(id=7, name=user_1, age=4)
User(id=15, name=user_9, age=14)
User(id=16, name=user_10, age=5)
User(id=19, name=user_13, age=12)
User(id=20, name=user_14, age=12)
User(id=22, name=user_16, age=5)
User(id=35, name=user_29, age=13)
User(id=36, name=user_30, age=0)
User(id=40, name=user_34, age=9)
User(id=50, name=user_44, age=0)
User(id=73, name=user_67, age=4)
User(id=79, name=user_73, age=12)
User(id=80, name=user_74, age=16)
User(id=83, name=user_77, age=9)
User(id=91, name=user_85, age=10)
User(id=92, name=user_86, age=12)
User(id=99, name=user_93, age=11)
User(id=105, name=user_99, age=8)


In [76]:
for u in session.query(User).filter(User.age < 18).filter(User.age > 12):
    print(u)

2017-11-26 10:47:02,952 INFO sqlalchemy.engine.base.Engine SELECT user.id AS user_id, user.name AS user_name, user.age AS user_age 
FROM user 
WHERE user.age < %(age_1)s AND user.age > %(age_2)s
2017-11-26 10:47:02,954 INFO sqlalchemy.engine.base.Engine {'age_1': 18, 'age_2': 12}
User(id=15, name=user_9, age=14)
User(id=35, name=user_29, age=13)
User(id=80, name=user_74, age=16)


In [77]:
from sqlalchemy import or_, and_, not_, between

In [78]:
for u in session.query(User).filter(or_(and_(User.age > 18, User.age < 30), and_(User.age > 50, User.age < 70))):
    print(u)

2017-11-26 10:52:25,565 INFO sqlalchemy.engine.base.Engine SELECT user.id AS user_id, user.name AS user_name, user.age AS user_age 
FROM user 
WHERE user.age > %(age_1)s AND user.age < %(age_2)s OR user.age > %(age_3)s AND user.age < %(age_4)s
2017-11-26 10:52:25,566 INFO sqlalchemy.engine.base.Engine {'age_1': 18, 'age_2': 30, 'age_3': 50, 'age_4': 70}
User(id=1, name=miracle, age=20)
User(id=5, name=test, age=19)
User(id=8, name=user_2, age=24)
User(id=11, name=user_5, age=26)
User(id=12, name=user_6, age=29)
User(id=13, name=user_7, age=67)
User(id=18, name=user_12, age=24)
User(id=21, name=user_15, age=67)
User(id=25, name=user_19, age=55)
User(id=27, name=user_21, age=53)
User(id=28, name=user_22, age=57)
User(id=31, name=user_25, age=19)
User(id=39, name=user_33, age=56)
User(id=48, name=user_42, age=29)
User(id=51, name=user_45, age=63)
User(id=56, name=user_50, age=29)
User(id=57, name=user_51, age=61)
User(id=58, name=user_52, age=68)
User(id=59, name=user_53, age=26)
User(id=

In [83]:
for u in session.query(User).filter(User.name.like('%0%')):
    print(u.name)

2017-11-26 10:58:56,379 INFO sqlalchemy.engine.base.Engine SELECT user.id AS user_id, user.name AS user_name, user.age AS user_age 
FROM user 
WHERE user.name LIKE %(name_1)s
2017-11-26 10:58:56,380 INFO sqlalchemy.engine.base.Engine {'name_1': '%0%'}
user_0
user_10
user_20
user_30
user_40
user_50
user_60
user_70
user_80
user_90


In [84]:
for u in session.query(User).filter(User.age < 18).order_by(User.age):
    print(u)

2017-11-26 11:12:27,466 INFO sqlalchemy.engine.base.Engine SELECT user.id AS user_id, user.name AS user_name, user.age AS user_age 
FROM user 
WHERE user.age < %(age_1)s ORDER BY user.age
2017-11-26 11:12:27,467 INFO sqlalchemy.engine.base.Engine {'age_1': 18}
User(id=50, name=user_44, age=0)
User(id=36, name=user_30, age=0)
User(id=73, name=user_67, age=4)
User(id=7, name=user_1, age=4)
User(id=22, name=user_16, age=5)
User(id=16, name=user_10, age=5)
User(id=105, name=user_99, age=8)
User(id=40, name=user_34, age=9)
User(id=83, name=user_77, age=9)
User(id=91, name=user_85, age=10)
User(id=99, name=user_93, age=11)
User(id=92, name=user_86, age=12)
User(id=79, name=user_73, age=12)
User(id=19, name=user_13, age=12)
User(id=20, name=user_14, age=12)
User(id=35, name=user_29, age=13)
User(id=15, name=user_9, age=14)
User(id=80, name=user_74, age=16)


In [85]:
for u in session.query(User).filter(User.age < 18).order_by(User.age.desc()):
    print(u)

2017-11-26 11:13:24,682 INFO sqlalchemy.engine.base.Engine SELECT user.id AS user_id, user.name AS user_name, user.age AS user_age 
FROM user 
WHERE user.age < %(age_1)s ORDER BY user.age DESC
2017-11-26 11:13:24,683 INFO sqlalchemy.engine.base.Engine {'age_1': 18}
User(id=80, name=user_74, age=16)
User(id=15, name=user_9, age=14)
User(id=35, name=user_29, age=13)
User(id=19, name=user_13, age=12)
User(id=20, name=user_14, age=12)
User(id=79, name=user_73, age=12)
User(id=92, name=user_86, age=12)
User(id=99, name=user_93, age=11)
User(id=91, name=user_85, age=10)
User(id=40, name=user_34, age=9)
User(id=83, name=user_77, age=9)
User(id=105, name=user_99, age=8)
User(id=22, name=user_16, age=5)
User(id=16, name=user_10, age=5)
User(id=73, name=user_67, age=4)
User(id=7, name=user_1, age=4)
User(id=36, name=user_30, age=0)
User(id=50, name=user_44, age=0)


In [86]:
for u in session.query(User).filter(User.age < 18).order_by(User.age.asc()):
    print(u)

2017-11-26 11:13:59,134 INFO sqlalchemy.engine.base.Engine SELECT user.id AS user_id, user.name AS user_name, user.age AS user_age 
FROM user 
WHERE user.age < %(age_1)s ORDER BY user.age ASC
2017-11-26 11:13:59,136 INFO sqlalchemy.engine.base.Engine {'age_1': 18}
User(id=50, name=user_44, age=0)
User(id=36, name=user_30, age=0)
User(id=73, name=user_67, age=4)
User(id=7, name=user_1, age=4)
User(id=22, name=user_16, age=5)
User(id=16, name=user_10, age=5)
User(id=105, name=user_99, age=8)
User(id=40, name=user_34, age=9)
User(id=83, name=user_77, age=9)
User(id=91, name=user_85, age=10)
User(id=99, name=user_93, age=11)
User(id=92, name=user_86, age=12)
User(id=79, name=user_73, age=12)
User(id=19, name=user_13, age=12)
User(id=20, name=user_14, age=12)
User(id=35, name=user_29, age=13)
User(id=15, name=user_9, age=14)
User(id=80, name=user_74, age=16)


In [87]:
for u in session.query(User).filter(User.age < 18).order_by(User.age.desc()).order_by(User.id.asc()):
    print(u)

2017-11-26 11:14:27,173 INFO sqlalchemy.engine.base.Engine SELECT user.id AS user_id, user.name AS user_name, user.age AS user_age 
FROM user 
WHERE user.age < %(age_1)s ORDER BY user.age DESC, user.id ASC
2017-11-26 11:14:27,174 INFO sqlalchemy.engine.base.Engine {'age_1': 18}
User(id=80, name=user_74, age=16)
User(id=15, name=user_9, age=14)
User(id=35, name=user_29, age=13)
User(id=19, name=user_13, age=12)
User(id=20, name=user_14, age=12)
User(id=79, name=user_73, age=12)
User(id=92, name=user_86, age=12)
User(id=99, name=user_93, age=11)
User(id=91, name=user_85, age=10)
User(id=40, name=user_34, age=9)
User(id=83, name=user_77, age=9)
User(id=105, name=user_99, age=8)
User(id=16, name=user_10, age=5)
User(id=22, name=user_16, age=5)
User(id=7, name=user_1, age=4)
User(id=73, name=user_67, age=4)
User(id=36, name=user_30, age=0)
User(id=50, name=user_44, age=0)


In [88]:
for u in session.query(User).limit(5):
    print(u)

2017-11-26 11:15:29,944 INFO sqlalchemy.engine.base.Engine SELECT user.id AS user_id, user.name AS user_name, user.age AS user_age 
FROM user 
 LIMIT %(param_1)s
2017-11-26 11:15:29,945 INFO sqlalchemy.engine.base.Engine {'param_1': 5}
User(id=1, name=miracle, age=20)
User(id=5, name=test, age=19)
User(id=6, name=user_0, age=50)
User(id=7, name=user_1, age=4)
User(id=8, name=user_2, age=24)


In [89]:
for u in session.query(User).offset(1).limit(5):
    print(u)

2017-11-26 11:17:13,065 INFO sqlalchemy.engine.base.Engine SELECT user.id AS user_id, user.name AS user_name, user.age AS user_age 
FROM user 
 LIMIT %(param_1)s, %(param_2)s
2017-11-26 11:17:13,066 INFO sqlalchemy.engine.base.Engine {'param_1': 1, 'param_2': 5}
User(id=5, name=test, age=19)
User(id=6, name=user_0, age=50)
User(id=7, name=user_1, age=4)
User(id=8, name=user_2, age=24)
User(id=9, name=user_3, age=46)


In [91]:
len(list(session.query(User).filter(User.age < 18)))

2017-11-26 11:19:19,345 INFO sqlalchemy.engine.base.Engine SELECT user.id AS user_id, user.name AS user_name, user.age AS user_age 
FROM user 
WHERE user.age < %(age_1)s
2017-11-26 11:19:19,346 INFO sqlalchemy.engine.base.Engine {'age_1': 18}


18

In [92]:
session.query(User).filter(User.age < 18).count()

2017-11-26 11:20:14,412 INFO sqlalchemy.engine.base.Engine SELECT count(*) AS count_1 
FROM (SELECT user.id AS user_id, user.name AS user_name, user.age AS user_age 
FROM user 
WHERE user.age < %(age_1)s) AS anon_1
2017-11-26 11:20:14,413 INFO sqlalchemy.engine.base.Engine {'age_1': 18}


18

In [93]:
session.query(User).filter(User.age < 18).all()

2017-11-26 11:20:51,436 INFO sqlalchemy.engine.base.Engine SELECT user.id AS user_id, user.name AS user_name, user.age AS user_age 
FROM user 
WHERE user.age < %(age_1)s
2017-11-26 11:20:51,437 INFO sqlalchemy.engine.base.Engine {'age_1': 18}


In [94]:
list(session.query(User).filter(User.age < 18))

2017-11-26 11:21:12,280 INFO sqlalchemy.engine.base.Engine SELECT user.id AS user_id, user.name AS user_name, user.age AS user_age 
FROM user 
WHERE user.age < %(age_1)s
2017-11-26 11:21:12,282 INFO sqlalchemy.engine.base.Engine {'age_1': 18}


In [95]:
session.query(User).filter(User.age < 18).first()

2017-11-26 11:21:23,847 INFO sqlalchemy.engine.base.Engine SELECT user.id AS user_id, user.name AS user_name, user.age AS user_age 
FROM user 
WHERE user.age < %(age_1)s 
 LIMIT %(param_1)s
2017-11-26 11:21:23,848 INFO sqlalchemy.engine.base.Engine {'age_1': 18, 'param_1': 1}


In [96]:
session.query(User).filter(User.age < 18).limit(1).one()

2017-11-26 11:22:03,673 INFO sqlalchemy.engine.base.Engine SELECT user.id AS user_id, user.name AS user_name, user.age AS user_age 
FROM user 
WHERE user.age < %(age_1)s 
 LIMIT %(param_1)s
2017-11-26 11:22:03,675 INFO sqlalchemy.engine.base.Engine {'age_1': 18, 'param_1': 1}


In [108]:
session.query(User.name, User.age).filter(User.age < 30).first()

2017-11-26 11:32:22,927 INFO sqlalchemy.engine.base.Engine SELECT user.name AS user_name, user.age AS user_age 
FROM user 
WHERE user.age < %(age_1)s 
 LIMIT %(param_1)s
2017-11-26 11:32:22,928 INFO sqlalchemy.engine.base.Engine {'age_1': 30, 'param_1': 1}


('miracle', 20)

In [98]:
session.query(User).filter(User.age < 18).delete()

2017-11-26 11:22:58,865 INFO sqlalchemy.engine.base.Engine DELETE FROM user WHERE user.age < %(age_1)s
2017-11-26 11:22:58,867 INFO sqlalchemy.engine.base.Engine {'age_1': 18}


18

In [99]:
session.commit()

2017-11-26 11:24:08,704 INFO sqlalchemy.engine.base.Engine COMMIT


In [100]:
from sqlalchemy import func

In [101]:
session.query(User, func.count(User.id)).all()

2017-11-26 11:30:03,980 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2017-11-26 11:30:03,981 INFO sqlalchemy.engine.base.Engine SELECT user.id AS user_id, user.name AS user_name, user.age AS user_age, count(user.id) AS count_1 
FROM user
2017-11-26 11:30:03,982 INFO sqlalchemy.engine.base.Engine {}


[(<__main__.User at 0x10c7fab38>, 84)]

In [102]:
session.query(func.count(User.id)).all()

2017-11-26 11:30:29,118 INFO sqlalchemy.engine.base.Engine SELECT count(user.id) AS count_1 
FROM user
2017-11-26 11:30:29,120 INFO sqlalchemy.engine.base.Engine {}


[(84)]

In [103]:
session.query(func.count(User.id)).first()

2017-11-26 11:30:41,818 INFO sqlalchemy.engine.base.Engine SELECT count(user.id) AS count_1 
FROM user 
 LIMIT %(param_1)s
2017-11-26 11:30:41,819 INFO sqlalchemy.engine.base.Engine {'param_1': 1}


(84)

In [104]:
session.query(func.max(User.age)).first()

2017-11-26 11:30:52,774 INFO sqlalchemy.engine.base.Engine SELECT max(user.age) AS max_1 
FROM user 
 LIMIT %(param_1)s
2017-11-26 11:30:52,775 INFO sqlalchemy.engine.base.Engine {'param_1': 1}


(98)

In [105]:
session.query(func.avg(User.age)).first()

2017-11-26 11:30:59,225 INFO sqlalchemy.engine.base.Engine SELECT avg(user.age) AS avg_1 
FROM user 
 LIMIT %(param_1)s
2017-11-26 11:30:59,227 INFO sqlalchemy.engine.base.Engine {'param_1': 1}


(Decimal('54.5952'))

In [106]:
session.query(User.age, func.count(User.id)).group_by(User.age).all()

2017-11-26 11:31:33,734 INFO sqlalchemy.engine.base.Engine SELECT user.age AS user_age, count(user.id) AS count_1 
FROM user GROUP BY user.age
2017-11-26 11:31:33,735 INFO sqlalchemy.engine.base.Engine {}


[(18, 3),
 (19, 2),
 (20, 1),
 (23, 1),
 (24, 2),
 (26, 2),
 (28, 2),
 (29, 3),
 (32, 1),
 (34, 1),
 (35, 3),
 (36, 4),
 (40, 3),
 (42, 1),
 (44, 2),
 (45, 1),
 (46, 2),
 (47, 1),
 (49, 2),
 (50, 2),
 (53, 2),
 (55, 1),
 (56, 1),
 (57, 1),
 (58, 2),
 (59, 1),
 (60, 1),
 (61, 3),
 (63, 4),
 (67, 3),
 (68, 2),
 (70, 1),
 (71, 1),
 (72, 2),
 (73, 1),
 (76, 1),
 (78, 1),
 (79, 2),
 (80, 1),
 (81, 2),
 (82, 1),
 (83, 1),
 (84, 1),
 (85, 2),
 (87, 1),
 (91, 1),
 (92, 2),
 (94, 1),
 (95, 1),
 (98, 1)]

In [3]:
from sqlalchemy import ForeignKey, ForeignKeyConstraint
from sqlalchemy.orm import relationship
from sqlalchemy import Column, String, Integer
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine
import random

In [4]:
Base = declarative_base()
engine = create_engine('mysql+pymysql://root@127.0.0.1:3306/demo', echo=True)
Session = sessionmaker(bind=engine)
session = Session()

In [6]:
class Author(Base):
    __tablename__ = 'author'
    
    id = Column(Integer, primary_key=True, autoincrement=True)
    name = Column(String(64), unique=True, nullable=False)
    
    posts = relationship('Post')
    
    def __str__(self):
        return 'Author<id={}, name={}'.format(self.id, self.name)
    
    def __repr__(self):
        return 'Author<id={}, name={}'.format(self.id, self.name)

In [5]:
class Post(Base):
    __tablename__ = 'post'
    
    id = Column(Integer, primary_key=True, autoincrement=True)
    title = Column(String(128), nullable=False, index=True)
    content = Column(String(8096), nullable=False)
    
    author_id = Column(Integer, ForeignKey('author.id'), nullable=False)
    
    author = relationship('Author')
    
    def __str__(self):
        return 'Post<id={}, title={}'.format(self.id, self.title)
    
    def __repr__(self):
        return 'Post<id={}, title={}'.format(self.id, self.title)

In [7]:
Base.metadata.create_all(engine)

2017-11-26 11:47:06,713 INFO sqlalchemy.engine.base.Engine SHOW VARIABLES LIKE 'sql_mode'
2017-11-26 11:47:06,714 INFO sqlalchemy.engine.base.Engine {}
2017-11-26 11:47:06,716 INFO sqlalchemy.engine.base.Engine SELECT DATABASE()
2017-11-26 11:47:06,717 INFO sqlalchemy.engine.base.Engine {}
2017-11-26 11:47:06,718 INFO sqlalchemy.engine.base.Engine show collation where `Charset` = 'utf8' and `Collation` = 'utf8_bin'
2017-11-26 11:47:06,718 INFO sqlalchemy.engine.base.Engine {}
2017-11-26 11:47:06,720 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS CHAR(60)) AS anon_1
2017-11-26 11:47:06,721 INFO sqlalchemy.engine.base.Engine {}
2017-11-26 11:47:06,722 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS CHAR(60)) AS anon_1
2017-11-26 11:47:06,722 INFO sqlalchemy.engine.base.Engine {}
2017-11-26 11:47:06,724 INFO sqlalchemy.engine.base.Engine SELECT CAST('test collated returns' AS CHAR CHARACTER SET utf8) COLLATE utf8_bin AS anon_1
2017-11-26 11

In [8]:
author = Author()

In [10]:
author.name = 'miracle'

In [11]:
session.add(author)

In [12]:
session.commit()

2017-11-26 11:48:31,280 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2017-11-26 11:48:31,282 INFO sqlalchemy.engine.base.Engine INSERT INTO author (name) VALUES (%(name)s)
2017-11-26 11:48:31,283 INFO sqlalchemy.engine.base.Engine {'name': 'miracle'}
2017-11-26 11:48:31,284 INFO sqlalchemy.engine.base.Engine COMMIT


In [13]:
author

2017-11-26 11:48:41,731 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2017-11-26 11:48:41,732 INFO sqlalchemy.engine.base.Engine SELECT author.id AS author_id, author.name AS author_name 
FROM author 
WHERE author.id = %(param_1)s
2017-11-26 11:48:41,733 INFO sqlalchemy.engine.base.Engine {'param_1': 1}


Author<id=1, name=miracle

In [14]:
post= Post()

In [15]:
post.title = 'test'

In [16]:
post.content = 'test content'

In [17]:
post.author = author

In [18]:
session.add(post)

In [19]:
session.commit()

2017-11-26 11:49:38,572 INFO sqlalchemy.engine.base.Engine INSERT INTO post (title, content, author_id) VALUES (%(title)s, %(content)s, %(author_id)s)
2017-11-26 11:49:38,574 INFO sqlalchemy.engine.base.Engine {'title': 'test', 'content': 'test content', 'author_id': 1}
2017-11-26 11:49:38,575 INFO sqlalchemy.engine.base.Engine COMMIT


In [20]:
author.posts

2017-11-26 11:50:07,025 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2017-11-26 11:50:07,027 INFO sqlalchemy.engine.base.Engine SELECT author.id AS author_id, author.name AS author_name 
FROM author 
WHERE author.id = %(param_1)s
2017-11-26 11:50:07,028 INFO sqlalchemy.engine.base.Engine {'param_1': 1}
2017-11-26 11:50:07,030 INFO sqlalchemy.engine.base.Engine SELECT post.id AS post_id, post.title AS post_title, post.content AS post_content, post.author_id AS post_author_id 
FROM post 
WHERE %(param_1)s = post.author_id
2017-11-26 11:50:07,031 INFO sqlalchemy.engine.base.Engine {'param_1': 1}


[Post<id=1, title=test]

In [21]:
p = Post()

In [22]:
p.title = 'abc'

In [23]:
p.content = 'abc content'

In [24]:
a = Author()

In [25]:
a.name = 'young'

In [26]:
p.author = a

In [27]:
session.add(p)
session.commit()

2017-11-26 11:51:31,811 INFO sqlalchemy.engine.base.Engine INSERT INTO author (name) VALUES (%(name)s)
2017-11-26 11:51:31,813 INFO sqlalchemy.engine.base.Engine {'name': 'young'}
2017-11-26 11:51:31,815 INFO sqlalchemy.engine.base.Engine INSERT INTO post (title, content, author_id) VALUES (%(title)s, %(content)s, %(author_id)s)
2017-11-26 11:51:31,816 INFO sqlalchemy.engine.base.Engine {'title': 'abc', 'content': 'abc content', 'author_id': 2}
2017-11-26 11:51:31,817 INFO sqlalchemy.engine.base.Engine COMMIT


In [29]:
session.query(Post, Author).filter(Post.author_id == Author.id).filter(Author.name == 'miracle').all()

2017-11-26 11:53:00,184 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2017-11-26 11:53:00,186 INFO sqlalchemy.engine.base.Engine SELECT post.id AS post_id, post.title AS post_title, post.content AS post_content, post.author_id AS post_author_id, author.id AS author_id, author.name AS author_name 
FROM post, author 
WHERE post.author_id = author.id AND author.name = %(name_1)s
2017-11-26 11:53:00,187 INFO sqlalchemy.engine.base.Engine {'name_1': 'miracle'}


[(Post<id=1, title=test, Author<id=1, name=miracle)]

In [30]:
session.query(Post).join(Author).filter(Author.name == 'miracle').all()

2017-11-26 11:54:29,374 INFO sqlalchemy.engine.base.Engine SELECT post.id AS post_id, post.title AS post_title, post.content AS post_content, post.author_id AS post_author_id 
FROM post INNER JOIN author ON author.id = post.author_id 
WHERE author.name = %(name_1)s
2017-11-26 11:54:29,375 INFO sqlalchemy.engine.base.Engine {'name_1': 'miracle'}


[Post<id=1, title=test]